# Dodanie ścieżki do projektu

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importowanie bibliotek

In [2]:
from src.scrapers import healthdata, koronawirusunas, meteostat, policjapl, rmf24
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from src.utils import save, paths
from pathlib import Path

# Pobranie i zapisanie danych

In [3]:
update = False

if len(os.listdir(paths.get_path())) == 0 or update:
    data = {**healthdata.get_data(),
            **koronawirusunas.get_data(),
            **{"meteostat": pd.DataFrame(meteostat.get_data(end_date=str(datetime.datetime.now().date()), start_date="2020-01-01"))},
            **policjapl.scrape() 
           }
    save.save_dataframes(data=data)
else:
    print("Data directory is not empty. propably data inside.")

Data directory is not empty. propably data inside.


# Załadowanie danych

In [4]:
policja = pd.read_csv(Path(paths.get_path(), "policjapl.csv"), index_col=1, parse_dates=True, dayfirst=True)
health = pd.read_csv(Path(paths.get_path(), "healthdata.csv"), index_col=2, parse_dates=True, dayfirst=True)
korona = pd.read_csv(Path(paths.get_path(), "koronawirusunas.csv"), index_col=0, parse_dates=True, dayfirst=True)
meteo = pd.read_csv(Path(paths.get_path(), "meteostat.csv"), index_col=1, parse_dates=True, dayfirst=True)

# Czyszczenie danych

## Czyszczenie meteo

In [5]:
meteo.head()

,Unnamed: 0,temperature,temperature_min,temperature_max,precipitation,snowfall,snowdepth,winddirection,windspeed,peakgust,sunshine,pressure
date,,,,,,,,,,,,
2020-01-01,0,1.4,0.8,5.6,1.0,NaN,NaN,267.0,17.4,NaN,NaN,1029.3
2020-01-02,1,0.8,-1.7,3.1,NaN,NaN,NaN,NaN,13.8,NaN,NaN,1026.5
2020-01-03,2,0.5,-4.1,4.6,NaN,NaN,NaN,205.0,6.4,NaN,NaN,1020.1
2020-01-04,3,3.7,1.6,5.3,1.0,NaN,NaN,258.0,20.7,NaN,NaN,1012.3
2020-01-05,4,0.4,-2.4,2.3,1.8,NaN,NaN,283.0,17.9,NaN,NaN,1025.8


In [6]:
meteo.drop(["snowfall", "snowdepth", "peakgust", "sunshine","precipitation","Unnamed: 0"], axis=1, inplace=True)
meteo.index.name = None
meteo.head()

,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure
2020-01-01,1.4,0.8,5.6,267.0,17.4,1029.3
2020-01-02,0.8,-1.7,3.1,NaN,13.8,1026.5
2020-01-03,0.5,-4.1,4.6,205.0,6.4,1020.1
2020-01-04,3.7,1.6,5.3,258.0,20.7,1012.3
2020-01-05,0.4,-2.4,2.3,283.0,17.9,1025.8


In [7]:
meteo[meteo.isnull().any(axis=1)]

,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure
2020-01-02,0.8,-1.7,3.1,NaN,13.8,1026.5
2020-01-13,4.8,2.0,NaN,NaN,NaN,NaN
2020-05-03,10.4,6.0,14.2,NaN,11.4,1011.8


In [8]:
meteo['winddirection'].fillna(meteo['winddirection'].mean(), inplace = True)
meteo['temperature_max'].fillna(meteo['temperature_max'].mean(), inplace = True)
meteo['pressure'].fillna(meteo['pressure'].mean(), inplace = True)
meteo['windspeed'].fillna(meteo['windspeed'].mean(), inplace = True)

In [9]:
meteo[meteo.isnull().any(axis=1)]

,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure


## Czyszczenie health

In [10]:
health.columns

Index(['V1', 'location_name', 'allbed_mean', 'allbed_lower', 'allbed_upper',
       'ICUbed_mean', 'ICUbed_lower', 'ICUbed_upper', 'InvVen_mean',
       'InvVen_lower', 'InvVen_upper', 'deaths_mean', 'deaths_lower',
       'deaths_upper', 'admis_mean', 'admis_lower', 'admis_upper',
       'newICU_mean', 'newICU_lower', 'newICU_upper', 'totdea_mean',
       'totdea_lower', 'totdea_upper', 'bedover_mean', 'bedover_lower',
       'bedover_upper', 'icuover_mean', 'icuover_lower', 'icuover_upper',
       'deaths_mean_smoothed', 'deaths_lower_smoothed',
       'deaths_upper_smoothed', 'totdea_mean_smoothed',
       'totdea_lower_smoothed', 'totdea_upper_smoothed', 'mobility_data_type',
       'mobility_composite', 'total_tests_data_type', 'total_tests',
       'confirmed_infections', 'est_infections_mean', 'est_infections_lower',
       'est_infections_upper'],
      dtype='object')

In [11]:
health = health.drop(health[health.index > "2020-05-20"].index)
health = health[health.location_name == "Poland"]
health = health[["allbed_mean", "ICUbed_mean", "deaths_mean", "admis_mean", "newICU_mean", 
                 "totdea_mean", "bedover_mean", "icuover_mean", 
                 "mobility_composite", "total_tests", "confirmed_infections"]]
health.index.name = None

In [12]:
health.tail()

,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections
2020-05-16,503.262226,154.481105,8.00,62.113366,18.736346,915.000,0.0,0.0,-24.908228,20336.029457,322.594580
2020-05-17,490.272022,151.624111,10.00,61.066749,18.486367,925.000,0.0,0.0,-24.908228,20680.197429,317.566735
2020-05-18,482.775176,150.006594,11.00,59.620879,18.339989,936.000,0.0,0.0,-24.908228,20991.323394,326.231780
2020-05-19,478.116932,148.706070,12.00,57.895855,17.813176,948.000,0.0,0.0,-24.908228,21264.729779,336.145368
2020-05-20,472.177993,146.742048,15.03,55.608276,17.244547,971.487,0.0,0.0,-24.908228,21538.136164,NaN


In [13]:
health[health.isnull().any(axis=1)]

,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections
2019-12-13,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-14,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-15,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-16,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-17,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,5.601200,NaN,NaN
2020-03-01,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,6.314276,NaN,NaN
2020-03-02,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,6.925443,NaN,NaN
2020-03-03,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.0,7.350635,212.438138,NaN


In [14]:
health.fillna(0, inplace = True)

In [15]:
health[health.isnull().any(axis=1)]

,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections


## Czyszczenie policja

policja.head()

In [16]:
policja.drop(["Unnamed: 0"], axis=1, inplace=True)
policja.index.name = None
policja.head()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach
2020-05-23,16946,625,180,242,50,5,57
2020-05-22,17807,622,269,188,72,2,82
2020-05-21,16706,640,306,201,72,6,71
2020-05-20,16396,577,299,189,54,5,54
2020-05-20,16004,586,261,157,45,1,47


In [17]:
policja[policja.isnull().any(axis=1)]

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach


## Czyszczenie korona

In [18]:
korona.head()

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.53,4.70,NaN,NaN,NaN,NaN
2020-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.05,6.90,NaN,NaN,NaN,NaN
2020-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.24,4.46,NaN,NaN,NaN,NaN
2020-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.95,7.81,NaN,NaN,NaN,NaN
2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.57,22.21,NaN,NaN,NaN,NaN


In [19]:
korona.tail()

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-05-20,19739.0,674861.0,617967.0,19739.0,10594.0,962.0,8183.0,NaN,NaN,2414.0,76957.0,NaN,18984.0
2020-05-21,20143.0,697230.0,639523.0,20143.0,10719.0,972.0,8452.0,NaN,NaN,2330.0,77461.0,NaN,19729.0
2020-05-22,20619.0,719571.0,660800.0,20619.0,10906.0,982.0,8731.0,NaN,NaN,2261.0,77768.0,NaN,19500.0
2020-05-23,20931.0,750939.0,687080.0,20931.0,10961.0,993.0,8977.0,NaN,NaN,2271.0,80635.0,NaN,19836.0
2020-05-24,NaN,NaN,NaN,21236.0,11047.0,995.0,9194.0,NaN,NaN,2242.0,79326.0,NaN,19529.0


### Wypełnienie nan

In [20]:
korona['kwar_z'].fillna(0, inplace=True)
korona.fillna(method='ffill', inplace=True)
korona.fillna(0, inplace=True)
korona

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.53,4.70,0.0,0.0,0.0,0.0
2020-02-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.05,6.90,0.0,0.0,0.0,0.0
2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.24,4.46,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.95,7.81,0.0,0.0,0.0,0.0
2020-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.57,22.21,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-20,19739.0,674861.0,617967.0,19739.0,10594.0,962.0,8183.0,-39.59,-22.81,2414.0,76957.0,0.0,18984.0
2020-05-21,20143.0,697230.0,639523.0,20143.0,10719.0,972.0,8452.0,-39.59,-22.81,2330.0,77461.0,0.0,19729.0
2020-05-22,20619.0,719571.0,660800.0,20619.0,10906.0,982.0,8731.0,-39.59,-22.81,2261.0,77768.0,0.0,19500.0
2020-05-23,20931.0,750939.0,687080.0,20931.0,10961.0,993.0,8977.0,-39.59,-22.81,2271.0,80635.0,0.0,19836.0


In [21]:
korona[korona.isnull().any(axis=1)]

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor


## Łączenie danych

In [22]:
df = pd.merge(policja, meteo, how='outer', left_index=True, right_index=True)
df = pd.merge(df, health, how='outer', left_index=True, right_index=True)
df = pd.merge(df, korona, how='outer', left_index=True, right_index=True)

In [23]:
df.iloc[0]

Interwencje                        NaN
Zatrzymani na gorącym uczynku      NaN
Zatrzymani poszukiwani             NaN
Zatrzymani nietrzeźwi kierujący    NaN
Wypadki drogowe                    NaN
Zabici w wypadkach                 NaN
Ranni w wypadkach                  NaN
temperature                        NaN
temperature_min                    NaN
temperature_max                    NaN
winddirection                      NaN
windspeed                          NaN
pressure                           NaN
allbed_mean                        0.0
ICUbed_mean                        0.0
deaths_mean                        0.0
admis_mean                         0.0
newICU_mean                        0.0
totdea_mean                        0.0
bedover_mean                       0.0
icuover_mean                       0.0
mobility_composite                 0.0
total_tests                        0.0
confirmed_infections               0.0
smp                                NaN
testy                    

## Czyszczenie df

In [24]:
df.fillna(method='ffill', inplace=True)
df.head()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2019-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Usunięcie danych sprzed 01.01.2020

In [25]:
df.drop(df[df.index < "2020-01-01"].index, inplace = True)
df

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-01-01,19145.0,734.0,188.0,332.0,42.0,1.0,46.0,1.4,0.8,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,14654.0,513.0,333.0,166.0,60.0,5.0,67.0,0.8,-1.7,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,15507.0,468.0,405.0,122.0,62.0,3.0,69.0,0.5,-4.1,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,14528.0,435.0,176.0,135.0,59.0,4.0,64.0,3.7,1.6,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,11948.0,457.0,185.0,146.0,44.0,4.0,54.0,0.4,-2.4,2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-20,16004.0,586.0,261.0,157.0,45.0,1.0,47.0,11.2,6.0,17.0,...,19739.0,10594.0,962.0,8183.0,-39.59,-22.81,2414.0,76957.0,0.0,18984.0
2020-05-21,16706.0,640.0,306.0,201.0,72.0,6.0,71.0,10.1,5.0,15.0,...,20143.0,10719.0,972.0,8452.0,-39.59,-22.81,2330.0,77461.0,0.0,19729.0
2020-05-22,17807.0,622.0,269.0,188.0,72.0,2.0,82.0,10.3,2.0,17.0,...,20619.0,10906.0,982.0,8731.0,-39.59,-22.81,2261.0,77768.0,0.0,19500.0
2020-05-23,16946.0,625.0,180.0,242.0,50.0,5.0,57.0,12.6,4.0,18.0,...,20931.0,10961.0,993.0,8977.0,-39.59,-22.81,2271.0,80635.0,0.0,19836.0


In [26]:
df[df.isnull().any(axis=1)].head()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-01-01,19145.0,734.0,188.0,332.0,42.0,1.0,46.0,1.4,0.8,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,14654.0,513.0,333.0,166.0,60.0,5.0,67.0,0.8,-1.7,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,15507.0,468.0,405.0,122.0,62.0,3.0,69.0,0.5,-4.1,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,14528.0,435.0,176.0,135.0,59.0,4.0,64.0,3.7,1.6,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,11948.0,457.0,185.0,146.0,44.0,4.0,54.0,0.4,-2.4,2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.fillna(0,inplace=True)
df[df.isnull().any(axis=1)].head()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor


# Współcznniki korelacji

In [28]:
def highlight(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    l = []
    for v in s:
        if abs(v) > 0.85 and abs(v) != 1:
            l.append('background-color: red')
        elif abs(v) > 0.75 and abs(v) != 1:
            l.append('background-color: yellow')
        else: 
            l.append('')
    return l

## Pearson

In [29]:
cor_pearson = df.corr(method="pearson")
cor_pearson.style.apply(highlight)

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
Interwencje,1.000000,0.216793,0.035982,0.270364,0.205303,0.076302,0.173597,0.487500,0.396007,0.467060,-0.064273,-0.164547,-0.018093,0.401682,0.397885,0.393439,0.388574,0.391505,0.360863,nan,nan,-0.150038,0.377124,0.313240,0.379609,0.358651,0.293103,0.379655,0.415942,0.361297,0.324286,-0.088286,-0.055313,0.341911,0.317264,-0.265543,0.008268
Zatrzymani na gorącym uczynku,0.216793,1.000000,0.532484,0.646318,0.657419,0.203282,0.665597,0.010198,0.313240,-0.092312,0.169603,-0.085547,-0.332511,-0.244710,-0.217957,-0.225896,-0.355791,-0.336729,0.166174,nan,nan,0.734305,0.009312,-0.428413,0.096019,0.173710,0.299656,0.096273,0.052942,0.166829,0.274528,0.722303,0.789327,-0.373874,-0.523945,-0.521833,-0.788696
Zatrzymani poszukiwani,0.035982,0.532484,1.000000,-0.200950,0.711005,0.026653,0.699882,-0.313816,-0.034703,-0.383315,0.116673,0.035780,-0.293999,-0.483500,-0.472403,-0.363852,-0.545555,-0.534809,-0.248450,nan,nan,0.746370,-0.368972,-0.569068,-0.304766,-0.245990,-0.116368,-0.304750,-0.326430,-0.248709,-0.153173,0.739534,0.760133,-0.589606,-0.629040,-0.386323,-0.712990
Zatrzymani nietrzeźwi kierujący,0.270364,0.646318,-0.200950,1.000000,0.147175,0.214638,0.162083,0.272042,0.361177,0.225924,0.076141,-0.121624,-0.129559,0.134863,0.152425,0.042851,0.068970,0.081280,0.334873,nan,nan,0.187933,0.266037,-0.000570,0.309641,0.336098,0.352647,0.309946,0.297801,0.335902,0.360462,0.178496,0.223993,0.060883,-0.055766,-0.291355,-0.296086
Wypadki drogowe,0.205303,0.657419,0.711005,0.147175,1.000000,0.333233,0.970707,-0.081848,0.194229,-0.181633,0.087315,-0.096045,-0.366815,-0.387338,-0.370231,-0.343916,-0.468716,-0.455188,-0.074639,nan,nan,0.707363,-0.215877,-0.513054,-0.139838,-0.072820,0.054558,-0.139769,-0.175390,-0.074286,0.022863,0.719999,0.756711,-0.482085,-0.608271,-0.368699,-0.704047
Zabici w wypadkach,0.076302,0.203282,0.026653,0.214638,0.333233,1.000000,0.276446,0.002460,0.095126,-0.054247,0.008496,0.083096,-0.142334,-0.099878,-0.092684,-0.117563,-0.111795,-0.111455,-0.052719,nan,nan,0.163102,-0.098207,-0.117832,-0.070334,-0.066576,-0.049443,-0.070280,-0.056187,-0.052596,-0.053678,0.192140,0.174452,-0.139507,-0.184943,-0.045970,-0.180835
Ranni w wypadkach,0.173597,0.665597,0.699882,0.162083,0.970707,0.276446,1.000000,-0.120658,0.160954,-0.203640,0.077200,-0.105406,-0.360477,-0.398679,-0.380609,-0.347468,-0.489551,-0.473923,-0.087166,nan,nan,0.730309,-0.236457,-0.517802,-0.155195,-0.090938,0.041604,-0.155104,-0.185975,-0.086661,0.005505,0.744634,0.776264,-0.497813,-0.629053,-0.399036,-0.730904
temperature,0.487500,0.010198,-0.313816,0.272042,-0.081848,0.002460,-0.120658,1.000000,0.811908,0.964814,-0.043654,-0.107139,-0.214277,0.732755,0.739395,0.685951,0.684133,0.695319,0.708787,nan,nan,-0.493980,0.734031,0.620293,0.729482,0.681829,0.561161,0.729371,0.763068,0.708598,0.612163,-0.447928,-0.352681,0.729959,0.580946,-0.088466,0.284994
temperature_min,0.396007,0.313240,-0.034703,0.361177,0.194229,0.095126,0.160954,0.811908,1.000000,0.689614,0.093424,0.017570,-0.375080,0.475524,0.492636,0.457182,0.406938,0.418975,0.603898,nan,nan,-0.128589,0.558269,0.347257,0.588882,0.571212,0.542633,0.588647,0.596487,0.602941,0.553305,-0.098875,0.011644,0.440009,0.277897,-0.215940,-0.051012
temperature_max,0.467060,-0.092312,-0.383315,0.225924,-0.181633,-0.054247,-0.203640,0.964814,0.689614,1.000000,-0.064686,-0.122679,-0.162659,0.746021,0.748653,0.700444,0.707778,0.717557,0.676053,nan,nan,-0.575519,0.722524,0.656347,0.705562,0.651753,0.521614,0.705422,0.742798,0.675826,0.572622,-0.531111,-0.451781,0.764503,0.634006,-0

## Kendall

In [30]:
cor_pearson = df.corr(method="kendall")
cor_pearson.style.apply(highlight)

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
Interwencje,1.000000,0.130961,-0.008897,0.217859,0.135521,0.082443,0.115208,0.336292,0.271884,0.313355,-0.055150,-0.118685,-0.009081,0.229582,0.216355,0.267758,0.250400,0.245100,0.233491,nan,nan,-0.108953,0.241262,0.198996,0.244053,0.244144,0.260575,0.244039,0.344351,0.234262,0.345091,-0.070604,-0.064084,0.226668,0.228551,-0.237994,0.132694
Zatrzymani na gorącym uczynku,0.130961,1.000000,0.353108,0.479386,0.426803,0.135708,0.424031,-0.024410,0.151887,-0.080145,0.033776,-0.070805,-0.179390,-0.313229,-0.304697,-0.260818,-0.330793,-0.343115,-0.167155,nan,nan,0.517060,-0.164348,-0.383470,-0.159899,-0.159785,0.206984,-0.159890,0.019881,-0.166253,0.024080,0.513096,0.519421,-0.269486,-0.401086,-0.382630,-0.520112
Zatrzymani poszukiwani,-0.008897,0.353108,1.000000,-0.090089,0.535372,0.028789,0.523133,-0.179279,-0.010744,-0.238032,0.052197,0.011203,-0.203316,-0.416196,-0.415788,-0.320073,-0.433527,-0.444897,-0.341123,nan,nan,0.524083,-0.330044,-0.406748,-0.337811,-0.337792,-0.060026,-0.337792,-0.188865,-0.341586,-0.192076,0.579974,0.607932,-0.379191,-0.485674,-0.352005,-0.576437
Zatrzymani nietrzeźwi kierujący,0.217859,0.479386,-0.090089,1.000000,0.112636,0.187289,0.125897,0.177818,0.218547,0.153304,0.020740,-0.096697,-0.107133,0.017829,0.024669,0.004486,-0.009403,-0.004488,0.080169,nan,nan,0.167676,0.084549,-0.058174,0.086474,0.086574,0.263064,0.086469,0.228709,0.081076,0.231433,0.156502,0.132771,0.021959,-0.065456,-0.278621,-0.159385
Wypadki drogowe,0.135521,0.426803,0.535372,0.112636,1.000000,0.218857,0.861406,-0.058790,0.101498,-0.129594,0.003820,-0.055023,-0.244894,-0.354149,-0.353398,-0.332742,-0.382179,-0.390588,-0.287732,nan,nan,0.490061,-0.261942,-0.397062,-0.265231,-0.265321,0.034074,-0.265215,-0.127764,-0.287483,-0.129255,0.559734,0.559324,-0.317098,-0.456719,-0.324811,-0.520835
Zabici w wypadkach,0.082443,0.135708,0.028789,0.187289,0.218857,1.000000,0.186326,0.008415,0.063763,-0.032552,0.025919,0.081792,-0.103900,-0.069739,-0.065379,-0.090837,-0.082312,-0.080025,-0.082272,nan,nan,0.111646,-0.073617,-0.066169,-0.067309,-0.067416,-0.005585,-0.067305,-0.040118,-0.081858,-0.040378,0.135248,0.122982,-0.061779,-0.113943,-0.033341,-0.124774
Ranni w wypadkach,0.115208,0.424031,0.523133,0.125897,0.861406,0.186326,1.000000,-0.096336,0.071921,-0.152877,-0.007351,-0.062936,-0.239403,-0.375490,-0.371423,-0.346392,-0.406815,-0.416549,-0.305649,nan,nan,0.523732,-0.275861,-0.402902,-0.281509,-0.281388,0.023410,-0.281493,-0.140780,-0.304839,-0.143017,0.576741,0.575304,-0.327460,-0.472951,-0.333629,-0.537051
temperature,0.336292,-0.024410,-0.179279,0.177818,-0.058790,0.008415,-0.096336,1.000000,0.629663,0.832500,-0.022638,-0.061754,-0.160662,0.513936,0.512488,0.498731,0.503220,0.495993,0.530857,nan,nan,-0.282511,0.545507,0.425073,0.542463,0.542326,0.436566,0.542431,0.570089,0.530940,0.572565,-0.252286,-0.277034,0.537179,0.433819,-0.060719,0.327518
temperature_min,0.271884,0.151887,-0.010744,0.218547,0.101498,0.063763,0.071921,0.629663,1.000000,0.505573,0.057777,0.007142,-0.259884,0.287967,0.293716,0.283991,0.279635,0.265439,0.334957,nan,nan,-0.086193,0.344959,0.200747,0.343644,0.343519,0.436556,0.343624,0.445496,0.333690,0.445991,-0.067133,-0.073274,0.327226,0.212019,-0.169426,0.083147
temperature_max,0.313355,-0.080145,-0.238032,0.153304,-0.129594,-0.032552,-0.152877,0.832500,0.505573,1.000000,-0.039654,-0.076008,-0.110974,0.553830,0.552182,0.534259,0.532593,0.536566,0.557333,nan,nan,-0.336571,0.572918,0.473552,0.570480,0.570342,0.413626,0.570447,0.556834,0.557609,0.557082,-0.316535,-0.339846,0.573

## Pearson

In [31]:
cor_pearson = df.corr(method="spearman")
cor_pearson.style.apply(highlight)

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
Interwencje,1.000000,0.188920,0.040891,0.298087,0.200174,0.117575,0.168218,0.488731,0.398779,0.462460,-0.065173,-0.172135,-0.022799,0.336896,0.323618,0.389715,0.358301,0.350913,0.358783,nan,nan,-0.143798,0.367329,0.283534,0.369516,0.369560,0.327902,0.369516,0.472704,0.358954,0.473729,-0.083971,-0.079411,0.343022,0.299628,-0.292127,0.135456
Zatrzymani na gorącym uczynku,0.188920,1.000000,0.513704,0.655657,0.610137,0.186939,0.605582,-0.028820,0.223190,-0.120267,0.055730,-0.100727,-0.267944,-0.454449,-0.444603,-0.377490,-0.472155,-0.490639,-0.281750,nan,nan,0.711916,-0.277079,-0.540026,-0.273037,-0.272988,0.253491,-0.273036,-0.009646,-0.281295,-0.007470,0.692847,0.694444,-0.411379,-0.550902,-0.469186,-0.668707
Zatrzymani poszukiwani,0.040891,0.513704,1.000000,-0.103768,0.736807,0.044694,0.723518,-0.266135,-0.003598,-0.359026,0.074030,0.015049,-0.285541,-0.587235,-0.587879,-0.472982,-0.593658,-0.613233,-0.529295,nan,nan,0.708414,-0.515460,-0.589187,-0.522351,-0.522351,-0.084570,-0.522351,-0.285546,-0.530018,-0.287863,0.754942,0.780006,-0.552658,-0.656711,-0.428682,-0.743979
Zatrzymani nietrzeźwi kierujący,0.298087,0.655657,-0.103768,1.000000,0.174943,0.263417,0.191460,0.254893,0.320314,0.224376,0.028598,-0.142758,-0.165855,0.032065,0.040738,0.012382,-0.009274,-0.001197,0.110999,nan,nan,0.239744,0.117569,-0.076967,0.119858,0.119888,0.330564,0.119858,0.295090,0.111684,0.298326,0.217633,0.184826,0.041888,-0.088935,-0.343296,-0.221061
Wypadki drogowe,0.200174,0.610137,0.736807,0.174943,1.000000,0.304920,0.965452,-0.108079,0.150216,-0.207031,0.010281,-0.083470,-0.352752,-0.506139,-0.504880,-0.464227,-0.534581,-0.546414,-0.428517,nan,nan,0.671921,-0.402280,-0.554402,-0.405891,-0.405932,0.040435,-0.405890,-0.192646,-0.428493,-0.193054,0.720465,0.719709,-0.465098,-0.609162,-0.399668,-0.684203
Zabici w wypadkach,0.117575,0.186939,0.044694,0.263417,0.304920,1.000000,0.260898,0.012520,0.086884,-0.043619,0.035267,0.119441,-0.154783,-0.094079,-0.090767,-0.116463,-0.114958,-0.112140,-0.104177,nan,nan,0.159933,-0.095270,-0.092330,-0.086123,-0.086250,-0.003041,-0.086123,-0.045254,-0.103753,-0.045893,0.185332,0.170523,-0.081067,-0.160048,-0.038683,-0.171044
Ranni w wypadkach,0.168218,0.605582,0.723518,0.191460,0.965452,0.260898,1.000000,-0.155495,0.107067,-0.237724,-0.005205,-0.091843,-0.343560,-0.525550,-0.522985,-0.482006,-0.560419,-0.570918,-0.454861,nan,nan,0.699088,-0.424868,-0.558649,-0.430499,-0.430485,0.024032,-0.430498,-0.214890,-0.454570,-0.216305,0.737527,0.736139,-0.476278,-0.627125,-0.410674,-0.701113
temperature,0.488731,-0.028820,-0.266135,0.254893,-0.108079,0.012520,-0.155495,1.000000,0.820119,0.960040,-0.029320,-0.091285,-0.247693,0.699553,0.699542,0.672384,0.686281,0.682279,0.714961,nan,nan,-0.436464,0.728670,0.582250,0.725575,0.725542,0.555767,0.725574,0.747504,0.715130,0.748456,-0.392945,-0.409510,0.718611,0.601975,-0.074915,0.441127
temperature_min,0.398779,0.223190,-0.003598,0.320314,0.150216,0.086884,0.107067,0.820119,1.000000,0.703467,0.084669,0.006589,-0.381553,0.409982,0.419392,0.409703,0.393948,0.380161,0.466458,nan,nan,-0.126794,0.483894,0.294728,0.483497,0.483305,0.561815,0.483497,0.584578,0.466018,0.585112,-0.081332,-0.094509,0.460037,0.304138,-0.212479,0.107962
temperature_max,0.462460,-0.120267,-0.359026,0.224376,-0.207031,-0.043619,-0.237724,0.960040,0.703467,1.000000,-0.054256,-0.111597,-0.178259,0.743582,0.741240,0.710074,0.723409,0.728297,0.750269,nan,nan,-0.512166,0.765624,0.639139,0.762482,0.762463,0.527280,0.762481,0.736517,0.750418,0.736947,-0.482183,-0.501467,0.762

In [32]:
df.describe()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,...,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,14915.068493,507.506849,247.472603,162.143836,52.171233,5.006849,58.801370,6.358904,1.682192,10.732455,...,4659.842466,2976.417808,210.136986,1303.643836,-29.181986,-22.810205,1149.342466,49304.958904,10101.500000,15526.828767
std,1884.980701,112.324361,96.719587,46.182097,17.351283,2.805159,20.216286,4.339326,3.326688,5.641099,...,6692.568452,4193.176998,328.381079,2494.410692,34.338310,26.859398,1214.759368,56914.846455,34911.189485,17644.498642
min,9992.000000,254.000000,86.000000,79.000000,14.000000,0.000000,15.000000,-0.300000,-7.400000,0.500000,...,0.000000,0.000000,0.000000,0.000000,-84.100000,-80.220000,0.000000,0.000000,0.000000,0.000000
25%,13841.250000,420.250000,166.000000,130.500000,38.250000,3.000000,43.000000,2.800000,-0.575000,5.675000,...,0.000000,0.000000,0.000000,0.000000,-62.062500,-49.332500,0.000000,0.000000,0.000000,0.000000
50%,14760.000000,541.500000,222.000000,152.500000,54.000000,5.000000,61.000000,5.800000,1.550000,10.450000,...,86.000000,0.000000,2.500000,0.000000,-31.725000,-10.805000,523.500000,3782.000000,0.000000,16908.000000
75%,15977.000000,596.250000,341.000000,190.500000,65.000000,6.000000,73.750000,10.100000,4.000000,15.275000,...,9150.750000,7768.750000,356.750000,1025.250000,0.000000,0.000000,2472.500000,96677.250000,0.000000,21676.750000
max,19989.000000,753.000000,415.000000,332.000000,98.000000,15.000000,107.000000,17.500000,9.800000,25.400000,...,21236.000000,11047.000000,995.000000,9194.000000,43.570000,22.210000,3505.000000,183095.000000,169092.000000,59081.000000
